In [6]:
import html
import pandas as pd
import geopandas as gpd
import folium
from branca.element import Template, MacroElement
import gspread
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials# Define the scope of the application
from googleapiclient.discovery import build

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# Add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('autoscraper-380600-0d0c84856d6b.json', scope)

# Authorize the clientsheet 
client = gspread.authorize(creds)

sheet = client.open_by_key('1vxMnPSQyDxTy16mE2Oq2a1rH3vrBSruTU74f5xEWCB0')

# Drive client
drivesvc = build("drive", "v3", credentials=creds)

range_name = 'A1:AO200'

# --- Load polygon layer and make a safe center ---
gdf = gpd.read_file("M1-6.kml")
minx, miny, maxx, maxy = gdf.total_bounds
map_center = [(miny + maxy) / 2, (minx + minx + (maxx - minx)) / 2]  # same as (minx+maxx)/2

m = folium.Map(location=map_center, zoom_start=14, control_scale=True, tiles='CartoDBpositron')

# --- Add zoning polygon layer ---
folium.GeoJson(
    gdf,
    name="M1-6 Layer",
    style_function=lambda x: {
        "fillColor": "#BF77F6",
        "color": "#111827",
        "weight": 2,
        "fillOpacity": 0.35
    },
    tooltip=folium.GeoJsonTooltip(fields=[c for c in gdf.columns if c != "geometry"])
).add_to(m)

# --- Your points DataFrame (df) ---
def fetch_data(sheet, worksheet_name, range_name, df_name=None):
    print('Fetching data from Google Sheets...')
    worksheet = sheet.worksheet(worksheet_name)
    data = worksheet.get(range_name)
    df = pd.DataFrame(data)
    df.columns = df.iloc[0]  # Set first row as column headers
    df = df.drop(0).reset_index(drop=True)  # Drop the header row from the dataframe and reset index

    df.drop_duplicates(inplace=True)  # Drop duplicate rows
    print(f'Number of rows in {worksheet_name} worksheet: {len(df)}')
    return df

df = fetch_data(sheet, 'zoning_list_with_owners', range_name, 'df')

# Ensure coordinates are numeric
df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
df["lon"] = pd.to_numeric(df["lon"], errors="coerce")

# Keep rows with valid coordinates
points = df.dropna(subset=["lat", "lon"]).copy()

# If points exist, re-center on them
if not points.empty:
    m.location = [points["lat"].mean(), points["lon"].mean()]

# Columns to include in popup (omit geocoded/lat/lon)
popup_fields = [
    'Address', 'Owner Name', 'Existing Building & Use', 'Zoning Change',
    'Potential Residential Units', 'Potential Affordable Units', 'Potential Office Change',
    'Potential Retail Added','Potential Height'
]

def is_filled(val):
    if pd.isna(val):
        return False
    s = str(val).strip()
    return s != "" and s.lower() != "nan"

def fmt(val):
    # Try to pretty-format numbers; otherwise escape text
    try:
        if isinstance(val, (int, float)):
            return f"{val:,.0f}" if float(val).is_integer() else f"{val:,.2f}"
        s = str(val)
        # numeric-looking strings (avoid killing leading zeros)
        if s and s[0] != "0":
            raw = s.replace(",", "")
            n = float(raw)
            return f"{n:,.0f}" if n.is_integer() else f"{n:,.2f}"
    except Exception:
        pass
    return html.escape(str(val))

def build_popup_html(row):
    rows_html = []
    for col in popup_fields:
        if col in row.index and is_filled(row[col]):
            label = html.escape(col)
            value = fmt(row[col])
            rows_html.append(
                f"<tr>"
                f"<th style='text-align:left; padding:2px 8px; white-space:nowrap;'>{label}</th>"
                f"<td style='padding:2px 8px;'>{value}</td>"
                f"</tr>"
            )
    if not rows_html:
        rows_html.append("<tr><td>No details available</td></tr>")
    return (
        "<div style='font: 14px/1.35 -apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica,Arial,sans-serif;'>"
        "<table style='border-collapse:collapse;'>"
        + "".join(rows_html) +
        "</table>"
        "</div>"
    )

# ---- Status handling: PROPOSED vs PROJECTED (fallback: OTHER) ----
def normalize_status(val: str) -> str:
    if not is_filled(val):
        return "OTHER"
    s = str(val).strip().upper()
    if "PROPOSED" in s:
        return "PROPOSED"
    if "PROJECTED" in s:
        return "PROJECTED"
    # If your values are exactly "PROPOSED"/"PROJECTED", equality checks also work:
    # if s == "PROPOSED": return "PROPOSED"
    # if s == "PROJECTED": return "PROJECTED"
    return "OTHER"

# Colors per status (works well on CartoDB Positron)
status_colors = {
    "PROPOSED": "#90EE90",   # green
    "PROJECTED": "#45B6FE",  # blue
    "OTHER": "#6b7280"       # gray
}

# Create a FeatureGroup per status so they’re toggleable
fgs = {
    "PROPOSED": folium.FeatureGroup(name="POTENTIAL", show=True),
    "PROJECTED": folium.FeatureGroup(name="PROJECTED", show=True),
    # "OTHER": folium.FeatureGroup(name="OTHER", show=False),
}
for fg in fgs.values():
    fg.add_to(m)

# Keep counts for legend
counts = {"PROPOSED": 0, "PROJECTED": 0, "OTHER": 0}

# --- Add pins to their respective groups ---
for _, r in points.iterrows():
    status = normalize_status(r.get("Site", ""))
    color = status_colors[status]
    counts[status] += 1

    popup_html = build_popup_html(r)
    folium.CircleMarker(
        location=(float(r["lat"]), float(r["lon"])),
        radius=7,
        color="black",          # outline
        weight=1,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        popup=folium.Popup(popup_html, max_width=450)
    ).add_to(fgs[status])

# ---- Add a simple legend (counts update automatically) ----
legend_html = f"""
<div style="
  position: fixed; bottom: 18px; left: 18px; z-index: 9999;
  background: rgba(255,255,255,0.95); padding: 10px 12px; border: 1px solid #ddd; border-radius: 8px;
  font: 13px/1.3 -apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica,Arial,sans-serif;">
  <div style="font-weight:600; margin-bottom:6px;">Sites</div>
  <div style="display:flex; align-items:center; gap:6px; margin:2px 0;">
    <span style="width:12px; height:12px; background:{status_colors['PROPOSED']}; border:1px solid #111; display:inline-block; border-radius:50%;"></span>
    <span>POTENTIAL ({counts['PROPOSED']})</span>
  </div>
  <div style="display:flex; align-items:center; gap:6px; margin:2px 0;">
    <span style="width:12px; height:12px; background:{status_colors['PROJECTED']}; border:1px solid #111; display:inline-block; border-radius:50%;"></span>
    <span>PROJECTED ({counts['PROJECTED']})</span>
  </div>
  <div style="margin-top:6px; font-size:12px; color:#444;">Toggle layers in the top-right.</div>
</div>
"""
legend = MacroElement()
legend._template = Template(f"""{{% macro html(this, kwargs) %}}{legend_html}{{% endmacro %}}""")
m.get_root().add_child(legend)

# ---- Layer control for toggling ----
folium.LayerControl(collapsed=False).add_to(m)

# Save as index.html
m.save("index.html")
m


Fetching data from Google Sheets...
Number of rows in zoning_list_with_owners worksheet: 67


In [3]:
import os

base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/midtown_south_rezoning_08_28_25
